In [ ]:
from avalanche.benchmarks import RotatedMNIST

# Generate RotatedMNIST pseudo-tasks (e.g., 7 tasks with different rotations)
rotated_benchmark = RotatedMNIST(n_experiences=7, seed=1234)

# Print rotation details for each experience
for i, experience in enumerate(rotated_benchmark.train_stream):
    print(f"Experience {i+1} rotation: {experience.task_labels}")

In [ ]:
from avalanche.benchmarks import SplitMNIST

# Standard MNIST training
split_mnist = SplitMNIST(n_experiences=1, seed=1234)
model_base = SimpleMLP(num_classes=10).to(device)

trainer = Naive(
    model_base,
    optimizer=torch.optim.SGD(model_base.parameters(), lr=0.01),
    criterion=torch.nn.CrossEntropyLoss(),
    train_mb_size=128,
    device=device
)

# Pretrain on MNIST
trainer.train(split_mnist.train_stream[0], epochs=5)

In [ ]:
task_vectors = []  # To store pre-populated task vectors

for experience in rotated_benchmark.train_stream:
    model_tuned = SimpleMLP(num_classes=10).to(device)
    model_tuned.load_state_dict(model_base.state_dict())  # Start from base model

    # Fine-tune on rotated task
    trainer = Naive(
        model_tuned,
        optimizer=torch.optim.SGD(model_tuned.parameters(), lr=0.01),
        criterion=torch.nn.CrossEntropyLoss(),
        train_mb_size=128,
        device=device
    )
    trainer.train(experience, epochs=5)

    # Compute task vector
    task_vector = []
    for p_base, p_tuned in zip(model_base.parameters(), model_tuned.parameters()):
        task_vector.append((p_tuned.data - p_base.data).detach().cpu().numpy())
    task_vector = np.concatenate([p.flatten() for p in task_vector])

    # Store the task vector
    task_vectors.append(task_vector)

In [ ]:
import pickle

# Save pre-populated task vectors to a file
with open("rotated_task_vectors.pkl", "wb") as f:
    pickle.dump(task_vectors, f)

In [ ]:
# Load pre-populated task vectors
with open("rotated_task_vectors.pkl", "rb") as f:
    task_vectors_active = pickle.load(f)